# Level Break Out

### Load the data

In [61]:
from Utils.YahooDataCollector import YahooDataCollector
stock = YahooDataCollector('AstraMicro.NS')

[*********************100%%**********************]  1 of 1 completed


In [62]:
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats

df = stock.hist
df=df[df['Volume']!=0]
df.reset_index(inplace=True)

df['EMA'] = ta.ema(df.Close, length=150)
df.tail()

df=df[0:10000]

C:\Users\TI-SJL-0008\AppData\Local\Temp\ipykernel_29640\1167565012.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Trend detection

In [63]:
EMAsignal = [0]*len(df)
backcandles = 15

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.Open[i], df.Close[i])>=df.EMA[i]:
            dnt=0
        if min(df.Open[i], df.Close[i])<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        EMAsignal[row]=3
    elif upt==1:
        EMAsignal[row]=2
    elif dnt==1:
        EMAsignal[row]=1

df['EMASignal'] = EMAsignal

In [64]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].Low > df.iloc[i].Low:
            pivotLow=0
        if df.iloc[candle].High < df.iloc[i].High:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [65]:
window=10
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [66]:

    
import numpy as np
def pointpos(x):
    if x['isPivot']==1:
        return x['low']-1e-3
    elif x['isPivot']==2:
        return x['high']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

KeyError: 'low'

In [ ]:
dfpl = df[7800:8000]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
def detect_structure(candle, backcandles, window):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    if (candle <= (backcandles+window)) or (candle+window+1 >= len(df)):
        return 0
    
    localdf = df.iloc[candle-backcandles-window:candle-window] #window must be greater than pivot window to avoid look ahead bias
    highs = localdf[localdf['isPivot'] == 1].High.tail(3).values
    lows = localdf[localdf['isPivot'] == 2].Low.tail(3).values
    levelbreak = 0
    zone_width = 0.001
    if len(lows)==3:
        support_condition = True
        mean_low = lows.mean()
        for low in lows:
            if abs(low-mean_low)>zone_width:
                support_condition = False
                break
        if support_condition and (mean_low - df.loc[candle].Close)>zone_width*2:
            levelbreak = 1

    if len(highs)==3:
        resistance_condition = True
        mean_high = highs.mean()
        for high in highs:
            if abs(high-mean_high)>zone_width:
                resistance_condition = False
                break
        if resistance_condition and (df.loc[candle].Close-mean_high)>zone_width*2:
            levelbreak = 2
    return levelbreak

In [ ]:
#df['pattern_detected'] = df.index.map(lambda x: detect_structure(x, backcandles=40, window=15))
df['pattern_detected'] = df.apply(lambda row: detect_structure(row.name, backcandles=60, window=11), axis=1)

In [ ]:
df[df['pattern_detected']!=0]

,Date,Open,High,Low,Close,Adj Close,Volume,EMA,EMASignal,isPivot,pointpos,pattern_detected
3749,2020-01-24,87.000000,94.000000,86.150002,93.199997,90.749428,1140134,83.190541,0,0,NaN,2
3751,2020-01-28,91.900002,95.199997,91.050003,92.449997,90.019150,452366,83.415903,0,0,NaN,2
3752,2020-01-29,93.949997,102.000000,93.000000,99.650002,97.029846,2628737,83.630924,0,1,102.001,2
3753,2020-01-30,100.199997,100.900002,93.650002,95.900002,93.378448,765660,83.793428,2,0,NaN,2
3754,2020-01-31,96.750000,96.800003,92.650002,93.949997,91.479706,297627,83.927952,2,0,NaN,2
3756,2020-02-04,93.750000,98.000000,93.050003,96.800003,94.254784,889013,84.193486,2,0,NaN,2
3757,2020-02-05,98.500000,98.500000,95.000000,96.150002,93.621872,345240,84.351850,2,0,NaN,2
3758,2020-02-06,96.349998,97.500000,93.500000,94.400002,91.917885,157159,84.484938,2,0,NaN,2
3759,2020-02-07,94.000000,97.650002,92.650002,95.050003,92.550789,440938,84.624873,2,0,NaN,2
3760,2020-02-10,94.400002,100.250000,94.349998,98.050003,95.471916,510857,84.802689,2,0,NaN,2
